In [ ]:
import tensorflow as tf  # model
from tensorflow.keras import callbacks,models,layers,Sequential,metrics,losses,optimizers,applications  # model
import tensorflow_datasets as tfds  # for dataset

import numpy as np    # math Compuation
import matplotlib.pyplot as plt ## for ploting charts\

In [ ]:
from pathlib import Path
import PIL.Image as Image
import cv2
from pprint import pprint
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns

import os,shutil

In [ ]:
# !pip install kaggle

In [ ]:
# !mkdir .\datasets\kaggle
# !cp kaggle.json ~/.kaggle/

In [ ]:
# shutil.rmtree('datasets/Saved_Model')
# shutil.rmtree('WorkingFolder')

In [ ]:
# import os
# os.environ['KAGGLE_USERNAME'] = 'prajwalsharma123'
# os.environ['KAGGLE_KEY']='6719f9750ed948ff6d82400eab62fe8c'

In [ ]:
# !kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification

In [ ]:
# import zipfile

# zip_path = 'eye-diseases-classification.zip'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall('datasets')

# os.remove(zip_path)

In [ ]:
data_directory = 'datasets/dataset/'
class_names = ['normal','glaucoma','cataract','diabetic_retinopathy'] ## Name should be same name of classs in folder directory 
class_short_names = ['normal','glaucoma','cataract','diabetic'] 

dataset_configuration={
    '_batch_size':16,
    '_seed' : 42,
    
    '_train_split':0.8,
    '_val_split':0.1,
    '_test_split':0.1,
    
    '_image_shape':(128,128,),
    '_n_images':4217,
    '_n_class':4,
}

general_configuration = {
    "_n_epochs": 50,
    "_learning_rate": 0.01,
}

classifier_configuration={
    '_n_Dense1':128,
    '_n_Dense2':16,
    "_dropout_rate":0.15,
    '_n_class':4,
}

## > Data Collection

In [ ]:
Dataset = tf.keras.utils.image_dataset_from_directory(
    directory = data_directory,
    labels = 'inferred', ## Labels will be generaed frm directory structure.
    label_mode = 'categorical',
    class_names = class_names, ## Not Neccessary As such .
    color_mode = 'rgb',
    batch_size = None,
    image_size = dataset_configuration['_image_shape'],
    shuffle = True,
    seed = dataset_configuration['_seed'], ## TO have same shuffling

    # validation_split=0.2, ## train_data 
    # subset='training', # as we setting vallidation split .
) 

In [ ]:
def SplitData(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO,SIZE):
    train_size = int(SIZE*TRAIN_RATIO)
    val_size = int(SIZE*VAL_RATIO)
    test_size = int(SIZE - train_size - val_size)
    
    dataset_configuration['_train_size'] = train_size
    dataset_configuration['_val_size'] = val_size
    dataset_configuration['_test_size'] = test_size
    
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size+val_size)
    
    return train_dataset,val_dataset,test_dataset

In [ ]:
TRAIN_RATIO = dataset_configuration['_train_split']
VAL_RATIO = dataset_configuration['_val_split']
TEST_RATIO = dataset_configuration['_test_split']
SIZE = dataset_configuration['_n_images']

train_dataset ,val_dataset,test_dataset =  SplitData(Dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO,SIZE)

In [ ]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

In [ ]:
for i in val_dataset.take(1):
    print(i)

In [ ]:
plt.figure(figsize=(16,8),facecolor='lightgrey')
for i,(image,label) in enumerate(train_dataset.take(32),1):
    ax = plt.subplot(4,8,i)
    plt.imshow(image/255.)
    plt.title(class_names[np.argmax(label)][:8],fontsize=10)
#     i+=1
    plt.axis('off')

## Data Augmentation

### > Layers

In [ ]:
augment_layers = models.Sequential([
    layers.RandomFlip(mode = 'horizontal'),
#     layers.RandomRotation(factor=(-0.025,0.25)), #-9 degree to 9 degree
    layers.RandomZoom(height_factor=(-0.2,0.2),width_factor=(-0.3,0.3),fill_mode='constant',fill_value=0),
    layers.RandomContrast(factor=0.6),
],name='augment_layers')

In [ ]:
# def augment(image,label):
#     return augment_layers(image,training=True),label

In [ ]:
image,label = next(iter(train_dataset))

plt.figure(figsize=(3,3))
plt.imshow(image/255)
plt.axis('off')
plt.title('Previous Image')
plt.show()

In [ ]:
plt.figure(figsize=(12,5)).suptitle('Augmented Image for the previous Image',fontsize = 16)
for i in range(1,25,1):
    aug_image = augment_layers(tf.expand_dims(image,0))[0]
    
    plt.subplot(3,8,i)
    plt.imshow(aug_image/255)
#     plt.imshow(image)

    plt.axis('off')
    plt.tight_layout()

## Data Preparation

In [ ]:
train_dataset=(
    train_dataset
#     .map(augment,num_parallel_calls=tf.data.AUTOTUNE) ## ony for train.
    .batch(dataset_configuration['_batch_size'])
    .prefetch(tf.data.AUTOTUNE)
)
train_dataset

In [ ]:
val_dataset=(
    val_dataset
    .batch(dataset_configuration['_batch_size'])
    .prefetch(tf.data.AUTOTUNE)
)
val_dataset

In [ ]:
X_test=[]
Y_test=[]

for images,labels in test_dataset:
    Y_test.append(labels.numpy())  # contain batches
    X_test.append(images.numpy())

In [ ]:
X_test[0].shape,Y_test[0].shape

In [ ]:
# X_test = tf.concat([ tf.reshape(X_test[:-1],shape=(-1,)+image_shape+(3,) ) , X_test[-1:][0] ],axis=0)
# Y_test = tf.concat()

In [ ]:
# image_shape = configuration['_image_shape']
# resize_rescale_layers = models.Sequential([
#     layers.Resizing( image_shape[0],image_shape[1]),
#     layers.Rescaling(1./255),
# ])

## Feature Extaction Models

In [ ]:
class CustomConv2D(layers.Layer):
    def __init__(self,n_filters,kernel_size,n_strides,padding='same',name = 'custom_conv2D'):
        super(CustomConv2D,self).__init__(name=name)

        self.conv = layers.Conv2D(
            filters = n_filters,
            kernel_size = kernel_size,
            activation = 'relu',
            strides = n_strides,
            padding = padding, 
        )
        self.batch_norm = layers.BatchNormalization()
    
    def call(self,x,training=True):
        x = self.conv(x)
        x = self.batch_norm(x,training = training)

        return x

### > Resnet Model

#### Resudual Block

In [ ]:
class ResidualBlock(layers.Layer):
    def  __init__(self,n_channels,n_strides=1):
        super(ResidualBlock,self).__init__(name='res_block')

        self.dotted = (n_strides !=1)# if dimention are not same befor and after block . 

        self.custom_conv_1 = CustomConv2D(n_channels,3,n_strides)
        self.custom_conv_2 = CustomConv2D(n_channels,3,1)
        self.activation = layers.Activation('relu')

        if(self.dotted):
            self.custom_conv_3 = CustomConv2D(n_channels,1,n_strides)
            ## To increse number of channels  size by 1.
    
    def call(self,input,training=None ):
        x = self.custom_conv_1(input,training = training)
        x = self.custom_conv_2(x,training = training)

        ## Submissiong The output ..
        if self.dotted:
            x_add = self.custom_conv_3(input,training = training)
            x_add = layers.Add()([x,x_add])
        else:  ## if dimentions are same.
            x_add = layers.Add()([x,input])

        return self.activation(x_add)
        

In [ ]:
image_shape = dataset_configuration['_image_shape']
class ResNet34(models.Model):
    def __init__(self):
        super(ResNet34,self).__init__(name='resnet_34')
        
        self.input_1 = layers.InputLayer(shape = image_shape+(3,),name = 'Input'),
        self.conv_1 = CustomConv2D(64,7,2,padding='same')
        self.max_pool = layers.MaxPooling2D(3,2,padding = 'same',)
   
        self.conv_2_1 = ResidualBlock(64)
        self.conv_2_2 = ResidualBlock(64)
        self.conv_2_3 = ResidualBlock(64)

        self.conv_3_1 = ResidualBlock(128,2) ## stride = 2 to downsample our feature.
        self.conv_3_2 = ResidualBlock(128)
        self.conv_3_3 = ResidualBlock(128)
        self.conv_3_4 = ResidualBlock(128)
        
        self.conv_4_1 = ResidualBlock(256,2)
        self.conv_4_2 = ResidualBlock(256)
        self.conv_4_3 = ResidualBlock(256)
        self.conv_4_4 = ResidualBlock(256)
        self.conv_4_5 = ResidualBlock(256)
        self.conv_4_6 = ResidualBlock(256)
        
        self.conv_5_1 = ResidualBlock(512,2)
        self.conv_5_2 = ResidualBlock(512)
        self.conv_5_3 = ResidualBlock(512)

        self.global_pool = layers.GlobalAveragePooling2D(name = 'Output')

    def call(self,x,training=None):

        x = self.conv_1(x,training = training)
        x = self.max_pool(x)

        x = self.conv_2_1(x,training = training)
        x = self.conv_2_2(x,training = training)
        x = self.conv_2_3(x,training = training)
        
        x = self.conv_3_1(x,training = training)
        x = self.conv_3_2(x,training = training)
        x = self.conv_3_3(x,training = training)
        x = self.conv_3_4(x,training = training)
        
        x = self.conv_4_1(x,training = training)
        x = self.conv_4_2(x,training = training)
        x = self.conv_4_3(x,training = training)
        x = self.conv_4_4(x,training = training)
        x = self.conv_4_5(x,training = training)
        x = self.conv_4_6(x,training = training)
    
        x = self.conv_5_1(x,training = training)
        x = self.conv_5_2(x,training = training)
        x = self.conv_5_3(x,training = training)

        output = self.global_pool(x)
        return output
        

In [ ]:
resnet_34 = ResNet34()

image_shape = dataset_configuration['_image_shape']
shape = (1,)+image_shape+(3,) 

resnet_34(tf.zeros(shape),training = False)
resnet_34.summary()

### Lenet

In [ ]:
def get_lenet_base_model():
    n_filter1 = 16
    n_filter2 = 6
    kernel_size = 3
    strides = 1
    pool_size = 2
    pool_strides = 2
    
    drop_rate = classifier_configuration['_dropout_rate']
    image_shape = dataset_configuration['_image_shape']
    
    model = models.Sequential([
        
        layers.InputLayer(shape = image_shape+(3,)),

        CustomConv2D(n_filter1 ,kernel_size ,strides , name='Conv_Norm_1'),
        layers.MaxPool2D( pool_size = pool_size, 
                         strides = pool_strides, 
                         name = "MaxPool1"),
        layers.Dropout(drop_rate,name = "Dropout1"),
        
        CustomConv2D(n_filter2,kernel_size,strides , name = 'Conv_Norm_2'),
        layers.MaxPool2D( pool_size = pool_size, 
                         strides = pool_strides , 
                         name = "MaxPool2"),
        layers.Dropout(drop_rate,name = "Dropout2"),
    
        layers.Flatten(name= "Flatten"),
        
    ],name = 'lenet_2')
    return model

In [ ]:
lenet_2 = get_lenet_base_model()
lenet_2.summary()

### > Other Models (Pretrainied)

In [ ]:
image_shape = dataset_configuration['_image_shape']
efficientnet_v2b0 = applications.efficientnet_v2.EfficientNetV2B0(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape= image_shape + (3,),
    include_top= False,
    pooling='avg',
    include_preprocessing=False
)
# efficientnet_v2m.trainable = True
efficientnet_v2b0.summary()

In [ ]:
image_shape = dataset_configuration['_image_shape']

vgg_16 = applications.vgg16.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=image_shape + (3,),
    include_top=False,
    pooling='avg',
)
# vgg_16.trainable = True
vgg_16.summary()

In [ ]:
image_shape = dataset_configuration['_image_shape']

inception_v3 = applications.inception_v3.InceptionV3(
    weights = 'imagenet',  # Load weights pre-trained on ImageNet.
    input_shape = image_shape + (3,),
    include_top=False,
    pooling='avg',
)

# inception_v3.trainable = True
inception_v3.summary()

In [ ]:
image_shape = dataset_configuration['_image_shape']

densenet_121 = applications.densenet.DenseNet121(
    weights = 'imagenet',  # Load weights pre-trained on ImageNet.
    input_shape = image_shape + (3,),
    include_top = False,
    pooling='avg',
)
# densenet_121.trainable = True
densenet_121.summary()

## Models CLassing 

In [ ]:
# def BuildModel(base_model,model_name ,training):
#     drop_rate = classifier_configuration['_dropout_rate']
#     n_unit_1 = classifier_configuration['_n_Dense1']
#     n_unit_2 = classifier_configuration['_n_Dense2']
#     n_class = classifier_configuration['_n_class']
#     image_shape = dataset_configuration['_image_shape']
    
#     ## -------------------------------------------------------------------------//
#     fn_input = layers.Input(shape = (None ,None,3),name = 'Input')
    
#     x = layers.Rescaling(1.0/255,name = 'Rescale')(fn_input)
#     x = layers.Resizing(height=image_shape[0],width = image_shape[1] ,name = 'Resize')(x)
#     x = augment_layers(x)
#     x = base_model(x , training = training)

#     x = layers.Dense(n_unit_1,activation = "relu",name = "Dense1")(x)
#     x = layers.BatchNormalization(name = 'Norm1')(x)
#     x = layers.Dropout(drop_rate,name = 'Dropout1')(x)
    
#     x = layers.Dense(n_unit_2,activation = "relu" , name = "Dense2")(x)
#     x = layers.BatchNormalization(name = 'Norm2')(x)
#     x = layers.Dropout(drop_rate, name = 'Dropout2')(x)
    
#     fn_output = layers.Dense(n_class,activation = "softmax", name = "Output")(x)
        
#     Model = models.Model(fn_input,fn_output,name = model_name)
#     ## --------------------------------------------------------------------------------//
#     return Model

In [ ]:
plateau_callback = callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.7, ## reduce by this factor . ## lr = lr*0.1
    patience=5, ## wait till these number of epochs ,
    verbose=1,
    mode='auto', ## similar to what studied above,,i..e Min or Max..
    min_delta=0.01, ## if change is less than delta than we will consider it as no improvement . 
    cooldown=0,  ## to wait after we have updated our lr ,,
    min_lr=0, ## dont go below this lr.
)

loss_function = losses.CategoricalCrossentropy()
# loss_function = losses.SparseCategoricalCrossentropy()

metrics_ = [metrics.CategoricalAccuracy(name='accuracy'),
           metrics.TopKCategoricalAccuracy(k=2,name ='top_k_accuracy'),
          ] 

def AddFromTo(History,history):
    for metric,values  in history.history.items():
        if(metric not in History.keys()):
            History[metric]=[]
        History[metric]+=values
    return History

In [ ]:
Base_logs_path ='WorkingFolder/logs/'
Model_path ='datasets/Saved_Model/{}.keras'
Model_weights_path = 'datasets/Saved_Model/{}_best.weights.h5'

Model_dict = {}
History_dict = {}
    
def ModelsPipeLine(model_name ,base_model ,is_pretrained=False ,training=True , lr=0.01 ,n_epoch=100):    
    n_epoch = n_epoch
    lr = lr
    History = {}
    model_log_path     = Base_logs_path + model_name
    model_weights_path = Model_weights_path.format(model_name)
    model_path         = Model_path.format(model_name)

    print(model_name.upper())
    ## -------------------------Buld Model --------------------------------//
#     BuildModel(base_model,model_name,training)
    ## Classifier Paramters --------------------------------//
    drop_rate = classifier_configuration['_dropout_rate']
    n_unit_1 = classifier_configuration['_n_Dense1']
    n_unit_2 = classifier_configuration['_n_Dense2']
    n_class = classifier_configuration['_n_class']
    image_shape = dataset_configuration['_image_shape']
    
    ## -------------------------------------------------------------------------//
    
    fn_input = layers.Input(shape = (None ,None,3),name = 'Input')
    
    x = layers.Rescaling(1.0/255,name = 'Rescale')(fn_input)
    x = layers.Resizing(height=image_shape[0],width = image_shape[1] ,name = 'Resize')(x)
    x = augment_layers(x)
    
    x = base_model(x , training = training)

    x = layers.Dense(n_unit_1,activation = "relu",name = "Dense1")(x)
    x = layers.BatchNormalization(name = 'Norm1')(x)
    x = layers.Dropout(drop_rate,name = 'Dropout1')(x)
    
    x = layers.Dense(n_unit_2,activation = "relu" , name = "Dense2")(x)
    x = layers.BatchNormalization(name = 'Norm2')(x)
    x = layers.Dropout(drop_rate, name = 'Dropout2')(x)
    
    fn_output = layers.Dense(n_class,activation = "softmax", name = "Output")(x)
        
    Model = models.Model(fn_input,fn_output,name = model_name)
    
    ## --------------------------------------------------------------//
    print(model_name +"Model Builded")
    
    ## Callbacks---------------------------------------------------------##
    
    checkpoint_callback = callbacks.ModelCheckpoint(  ## To save Best Models
        model_weights_path,
        monitor = 'val_accuracy',
        verbose = 0,
        save_best_only = True, ## will save best weights. 
        save_weights_only = True, ## If false will save whole model .. .
        mode = 'max', # if monitor val_loss that it will be min else if val_accuracy that it will be Max..
        save_freq = 'epoch', ## we will do this after every epoch .
    )
    
    tensorboard_callback = callbacks.TensorBoard(model_log_path)
    ## -TransferLearning-------------------------------------------------------##
    if(is_pretrained):
        print('TransferLearnining Begins')
        base_model.trainable = False
        n_epoch = int(n_epoch/2)
        
        Model.compile(
            optimizers.Adam(learning_rate=lr),
            loss = loss_function,
            metrics = metrics_,
        )
        history = Model.fit(
            train_dataset,
            validation_data= val_dataset,
            epochs = n_epoch,
            verbose=1,
            callbacks = [checkpoint_callback,
                         plateau_callback,
                         tensorboard_callback,
                        ]
        )
        History = AddFromTo(History,history)
        base_model.trainable = True
        lr = lr/10  
        
    #-----------------------------------------------------------------------#.
    print("Full Training begins")
    
    Model.compile(
        optimizers.Adam(learning_rate=lr),
        loss = loss_function,
        metrics = metrics_,
    )

    history = Model.fit(
        train_dataset,
        validation_data= val_dataset,
        epochs = n_epoch,
        verbose=1,
        callbacks = [checkpoint_callback,
                     plateau_callback,
                     tensorboard_callback,
                    ]
    )
    print('Saving Model and Results Begin')
    History = AddFromTo(History,history)
    
    Model.load_weights(model_weights_path)
    os.remove(model_weights_path)

    Model.save(model_path)  # now we have the best Model.

    Model_dict[model_name] = Model
    History_dict[model_name] = History  
        

In [ ]:
# # ## Model Dictionary have model name with  Model
# BaseModel_dict = {
#     'lenet_2' : lenet_2,
#     'resnet_34': resnet_34,
#     'inception_v3': inception_v3,
#     'densenet_121': densenet_121,
#     'vgg_16': vgg_16,
#     'efficientnet_v2m' :efficientnet_v2m,
# }

In [ ]:
model_name = 'efficientnet_v2b0'
base_model = efficientnet_v2b0
is_pretrained = True
training = False

ModelsPipeLine(model_name ,base_model ,is_pretrained ,training)

In [ ]:
model_name = 'resnet_34'
base_model = resnet_34
is_pretrained = False
training = True

ModelsPipeLine(model_name ,base_model ,is_pretrained ,training)

In [ ]:
Model_dict

In [ ]:
History_dict

In [ ]:

Model.load_weights(model_weights_path)
os.remove(model_weights_path)

Model.save(model_path)  # now we have the best Model.

Model_dict[model_name] = Model
History_dict[model_name] = History  

## Plots

In [ ]:
# History1 = history1.history
# History2 = history2.history

### > Comparision

In [ ]:
models_name_list = list(History_dict.keys())
metric_List = list(History_dict[models_name_list[0]].keys())

plt.figure(figsize = (12,14),facecolor='#beb068').suptitle("Model Performance Comparision",fontsize=16, y=1)

for i,metric in enumerate(metric_List,1):
    plt.subplot(4,2,i)
    
    for model_name in models_name_list:
        plt.plot(History_dict[model_name][metric],label= model_name,linewidth=1.5)
        
    plt.title(metric)
    plt.legend()
    plt.ylabel(metric + ' -->')
    plt.xlabel('epoch -->')
    plt.tight_layout()
    plt.grid()

### > Performance

In [ ]:
models_name_list = list(History_dict.keys())
metric_List = list(History_dict[models_name_list[0]].keys())

for model_name in models_name_list:
    plt.figure(figsize = (16,4),facecolor='#beb068').suptitle(model_name+" Performance",fontsize=16, y=1)
    for i,metric in enumerate(metric_List[:3],1):
        plt.subplot(1,3,i)
        plt.plot(History_dict[model_name][metric],label='train',linewidth=2)
        plt.plot(History_dict[model_name]['val_'+ metric],label='validation',linewidth=1.5)
        plt.title(metric)
        plt.legend()
        plt.ylabel(metric + ' -->')
        plt.xlabel('epoch -->')
        plt.subplots_adjust(wspace=0.2, hspace=0.3) 
        plt.grid()
    plt.show()

## Evaluation

In [ ]:
# Y_test =  tf.concat([np.argmax(Y_test[:-1],axis=-1).flatten(),np.argmax(Y_test[-1],axis=-1)],axis=0)
# Y_test

In [ ]:
# X_test = tf.concat([ tf.reshape(X_test[:-1],shape=(-1,)+image_shape+(3,) ) , X_test[-1:][0] ],axis=0)
# X_test.shape

In [ ]:
Y_true = np.argmax(Y_test , axis = -1)
Y_pred = effecient_model.predict(X_test,verbose=0) 
Y_pred = np.argmax(np.array(Y_pred),axis=-1

In [ ]:
models_name_list = list(History_dict.keys())
Y_pred_dict = {}
Y_true = np.argmax(Y_test , axis = -1)

for model_name, model in Model_dict.items():
    Y_pred = model.predict(X_test,verbose=0)    
    Y_pred_dict[model_name] = np.argmax(np.array(Y_pred),axis=-1)

### > Classification Report

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
models_name_list = list(History_dict.keys())

for model_name ,Y_pred in Y_pred_dict.items():
    print(model_name.upper())
    print(classification_report(Y_true,Y_pred))
    print('>-----------------------------------------------------------------------------------<','\n')

### > Confusion Matrix

In [ ]:
import seaborn as sn

plt.figure(figsize=(12,6),facecolor='#beb068').suptitle("Confusion Matrix",fontsize=16,y=0.95)

for i,(model_name,Y_pred) in enumerate(Y_pred_dict.items(),1):
    
    cn = tf.math.confusion_matrix(labels=Y_true,predictions=Y_pred)
    
    plt.subplot(2,3,i)
    sn.heatmap(cn ,annot=True , fmt = 'd',xticklabels=class_short_names ,yticklabels=class_short_names,)
    plt.xlabel("____Predicted____",fontsize=14)
    plt.ylabel("_____Truth______" ,fontsize=12)
    plt.yticks(rotation = 0)
    plt.tight_layout()
    plt.title(model_name)

## Testing

import matplotlib.pyplot as plt

fig = plt.figure(constrained_layout=True)

subfigs = fig.subfigures(2, 2)

for outerind, subfig in enumerate(subfigs.flat):
    subfig.suptitle(f'Subfig {outerind}')
    axs = subfig.subplots(2, 1)
    for innerind, ax in enumerate(axs.flat):
        ax.set_title(f'outer={outerind}, inner={innerind}', fontsize='small')
        ax.set_xticks([])
        ax.set_yticks([])
 
plt.show()

In [ ]:
Images = X_test[:16]
Labels = Y_true[:16]


model_list = [Model_dict['resnet_34'], Model_dict['lenet_2']]
model_names = ['resnet_34','lenet_2']
    
fig = plt.figure(figsize=(10,6),facecolor='lightgrey')

subfigs = fig.subfigures(1,len(model_list), wspace=0.1)

for subfig,model,model_name in zip(subfigs.flat,model_list,model_names):

    subfig.suptitle(model_name,y=1,fontsize=16)

    axs = subfig.subplots(4,4,)

    for i,ax in enumerate(axs.flat,0):
        ax.imshow(images[i]/255.)

        true = class_names[labels[i]]

        image = tf.expand_dims(images[i],axis=0)
        pred = model(image)[0]
#         pred = resnet_34(image)[0]
        pred = class_names[tf.argmax(pred,axis=-0).numpy()]

        ax.set_title(f'{true} : {pred}',fontsize=10)
        ax.axis('off')
        plt.subplots_adjust(hspace=0.0,wspace=0.0)
        plt.tight_layout()
fig.show()

## Saving Results

### > Model_History

In [ ]:
import json
s=json.dumps(book)

with open("History/History_dict.txt","wb") as file: 
    ## will create This directory along with file .. and will write s onto it ..
    file.write(s)

In [ ]:
# ## Reteriving 

# with open("History/History_dict.txt","r") as file: # ;'r' for reading 
#     History_str = f.read(); ## it is a JSON String
# History_dict_ = json.loads(History_str)
# History_dict_

### > Tensorboard

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir $Base_logs_path --port=6025

### >Models To WandB

In [ ]:
import os
# !wandb login --relogin 86e08e73408ad569e47977710a0d5253d116f899
!wandb login 86e08e73408ad569e47977710a0d5253d116f899
os.environ['WANDB_NOTEBOOK_NAME'] = 'Part10_MLOPS.ipynb'

In [ ]:
wandb.tensorboard.patch(root_logdir=Base_logs_path)

In [ ]:
import wandb
# from wandb.keras import WandbEvalCallback ,WandbCallback,WandbMetricsLogger, WandbModelCheckpoint

In [ ]:
wandb.init(project='EyeDiseaseDetection',
           entity='creater',
          )

In [ ]:
wandb.run

In [ ]:
wandb,finish()